# Add a file for tracking

`dvc add test/test_dag.gml`

# Creates a .dvc file

`data.xml.dvc`

# Ignore this in .igitignore

* `git add data/data.xml.dvc data/.gitignore`
* `git commit -m "Add raw data"`

# Upload DVC tracked data

```
dvc remote add -d storage s3://bucket/dvcstore
git add .dvc/config
git commit -m "Configure remote storage"
```

# Push

`dvc push`

# Pull

`dvc pull`

In [6]:
pd.DataFrame({'a': np.arange(10**6), 'b': np.repeat('hello', 10**6)}).to_parquet('test/dvc_dataframe_test.parquet')